<a href="https://colab.research.google.com/github/suhailphotos/ml4vfx/blob/fashionMNIST/notebook/fashionMNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import matplotlib.pyplot as plt

In [2]:
training_transform = transforms.Compose([
    transforms.RandomRotation(10),
    transforms.RandomAffine(0, translate=(0.1, 0.1)),
    transforms.ToTensor(),
])


training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=training_transform
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=transforms.ToTensor()
)

batch_size = 64
training_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

100.0%
100.0%
100.0%
100.0%


In [4]:
device = torch.device("cuda")
print(f"Using {device} device")

Using cuda device


In [5]:
class Net(nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    self.flatten = nn.Flatten()
    self.linear_relu_stack = nn.Sequential(
        nn.Linear(28*28, 512),
        nn.BatchNorm1d(512),
        nn.ReLU(),
        nn.Dropout(0.5),
        nn.Linear(512, 512),
        nn.BatchNorm1d(512),
        nn.ReLU(),
        nn.Dropout(0.5),
        nn.Linear(512, 10),
    )

  def forward(self, x):
    x = self.flatten(x)
    logits = self.linear_relu_stack(x)
    return logits

model = Net().to(device)
print(model)

Net(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.5, inplace=False)
    (4): Linear(in_features=512, out_features=512, bias=True)
    (5): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): Dropout(p=0.5, inplace=False)
    (8): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [6]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=2)

def train(dataloader, model, loss_fn, optimizer, scheduler):
    size = len(dataloader.dataset)
    model.train()
    running_loss = 0.0

    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        pred = model(X)
        loss = loss_fn(pred, y)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        running_loss += loss.item()

        if batch % 100 == 0:
            current = batch * len(X)
            print(f"loss: {loss.item():>7f}  [{current:>5d}/{size:>5d}]")

    avg_loss = running_loss / len(dataloader)
    scheduler.step(avg_loss)  # LR scheduler usually uses *average* loss

def test(dataloader, model, loss_fn, scheduler=None):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0

    with torch.no_grad():  # Better style than inference_mode() for compatibility
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    accuracy = correct / size

    if scheduler:  # Scheduler step only if passed in
        scheduler.step(test_loss)

    print(f"Test Error: \n Accuracy: {(100*accuracy):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [7]:
if __name__ == "__main__":

    epochs = 8

    for t in range(epochs):
        print(f"Epoch {t+1}\n-------------------------------")
        train(training_dataloader, model, loss_fn, optimizer, scheduler)
        test(test_dataloader, model, loss_fn, scheduler)

    print("Done!")

    torch.save(model.state_dict(), "fashion_mnist.pth")
    print("Saved PyTorch Model State")

Epoch 1
-------------------------------
loss: 2.438698  [    0/60000]
loss: 0.859127  [ 6400/60000]
loss: 0.724371  [12800/60000]
loss: 0.981122  [19200/60000]
loss: 0.819638  [25600/60000]
loss: 0.745696  [32000/60000]
loss: 0.718484  [38400/60000]
loss: 0.691018  [44800/60000]
loss: 0.812779  [51200/60000]
loss: 0.739286  [57600/60000]
Test Error: 
 Accuracy: 77.8%, Avg loss: 0.583719 

Epoch 2
-------------------------------
loss: 0.693540  [    0/60000]
loss: 0.704010  [ 6400/60000]
loss: 0.573834  [12800/60000]
loss: 0.774504  [19200/60000]
loss: 0.840871  [25600/60000]
loss: 0.733598  [32000/60000]
loss: 0.594600  [38400/60000]
loss: 0.762155  [44800/60000]
loss: 0.728860  [51200/60000]
loss: 0.675414  [57600/60000]
Test Error: 
 Accuracy: 80.3%, Avg loss: 0.528787 

Epoch 3
-------------------------------
loss: 0.666624  [    0/60000]
loss: 0.659431  [ 6400/60000]
loss: 0.493732  [12800/60000]
loss: 0.607879  [19200/60000]
loss: 0.707045  [25600/60000]
loss: 0.601005  [32000/600